# Deep Learning

## Assignment 3

Previously in `2_fullyconnected.ipynb`, we trained a logistic regression and a neural network model.

In this assignment, we're going to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`

In [2]:
pickle_file = 'data/notMNIST_refined.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Valid set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (197987, 28, 28) (197987,)
Valid set (9038, 28, 28) (9038,)
Test set (8811, 28, 28) (8811,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197987, 784) (197987, 10)
Validation set (9038, 784) (9038, 10)
Test set (8811, 784) (8811, 10)


In [31]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))) / predictions.shape[0]

### Problem 1

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, L2 loss for a tensor `t` can be computed using `nn.l2_loss(t)`. The right amount of the regularization should improve our validation / test accuracy.

Let's try with Linear Regression

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization for weights and normalized
        0.01 * tf.nn.l2_loss(weights)
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.261024
Minibatch accuracy: 11.7%
Validation accuracy: 10.6%
Minibatch loss at step 2000: 6.416584
Minibatch accuracy: 78.9%
Validation accuracy: 74.6%
Minibatch loss at step 4000: 1.753941
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 6000: 0.957863
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 8000: 0.766924
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 10000: 0.833300
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 12000: 0.677844
Minibatch accuracy: 88.3%
Validation accuracy: 81.4%
Minibatch loss at step 14000: 0.565475
Minibatch accuracy: 89.1%
Validation accuracy: 81.4%
Minibatch loss at step 16000: 0.954033
Minibatch accuracy: 75.8%
Validation accuracy: 81.6%
Minibatch loss at step 18000: 0.619791
Minibatch accuracy: 86.7%
Validation accuracy: 81.7%
Minibatch loss at step 20000: 0.750928
Minibatch accuracy: 80.5%
Validation

For my case, a learning rate $\alpha = 0.04$ and a L2 regularization normalized coefficient of $0.01$ would give the best test accuracy of $89.1\%$ 

** Let's give another try with a simple 1-hidden layer fully connected neural network **

In [7]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    out_weights = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    logits = tf.matmul(layer1, out_weights) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization
        0.001 * tf.nn.l2_loss(hidden_weights) + 
        0.001 * tf.nn.l2_loss(out_weights))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(
        tf.matmul(layer1_valid, out_weights) + out_biases)
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, out_weights) + out_biases)

In [8]:
num_steps = 40001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 4000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 718.394165
Minibatch accuracy: 7.0%
Validation accuracy: 17.1%
Minibatch loss at step 4000: 231.376785
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Minibatch loss at step 8000: 166.156906
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Minibatch loss at step 12000: 119.105927
Minibatch accuracy: 91.4%
Validation accuracy: 81.5%
Minibatch loss at step 16000: 86.579704
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 20000: 62.566208
Minibatch accuracy: 86.7%
Validation accuracy: 84.4%
Minibatch loss at step 24000: 45.593464
Minibatch accuracy: 85.2%
Validation accuracy: 84.1%
Minibatch loss at step 28000: 33.001579
Minibatch accuracy: 91.4%
Validation accuracy: 85.8%
Minibatch loss at step 32000: 23.959131
Minibatch accuracy: 93.0%
Validation accuracy: 86.9%
Minibatch loss at step 36000: 17.477411
Minibatch accuracy: 93.8%
Validation accuracy: 87.6%
Minibatch loss at step 40000: 12.831277
Minibatch accuracy: 9

In the above experiment, the best value for $\alpha = 0.04$ and L2 coefficient $0.001$ for the test accuracy to be $94.3\%$

### Problem 2

Let's demonstrate an extreme case of overfitting. Restrict the training data to just a few batches.

In [12]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    out_weights = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    logits = tf.matmul(layer1, out_weights) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization
        0.001 * tf.nn.l2_loss(hidden_weights) + 
        0.001 * tf.nn.l2_loss(out_weights))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(
        tf.matmul(layer1_valid, out_weights) + out_biases)
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, out_weights) + out_biases)

num_steps = 40001

# Use a smaller train_dataset
train_smaller = train_dataset[:384, :]
labels_smaller = train_labels[:384, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_smaller.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_smaller[offset:(offset + batch_size), :]
        batch_labels = labels_smaller[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 4000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 669.442017
Minibatch accuracy: 4.7%
Validation accuracy: 14.5%
Minibatch loss at step 4000: 228.910370
Minibatch accuracy: 100.0%
Validation accuracy: 51.6%
Minibatch loss at step 8000: 166.221985
Minibatch accuracy: 100.0%
Validation accuracy: 51.6%
Minibatch loss at step 12000: 120.701172
Minibatch accuracy: 100.0%
Validation accuracy: 51.7%
Minibatch loss at step 16000: 87.646507
Minibatch accuracy: 100.0%
Validation accuracy: 51.7%
Minibatch loss at step 20000: 63.644112
Minibatch accuracy: 100.0%
Validation accuracy: 51.8%
Minibatch loss at step 24000: 46.215027
Minibatch accuracy: 100.0%
Validation accuracy: 52.1%
Minibatch loss at step 28000: 33.559151
Minibatch accuracy: 100.0%
Validation accuracy: 53.0%
Minibatch loss at step 32000: 24.369371
Minibatch accuracy: 100.0%
Validation accuracy: 54.6%
Minibatch loss at step 36000: 17.696594
Minibatch accuracy: 100.0%
Validation accuracy: 57.0%
Minibatch loss at step 40000: 12.851608
Minibatch ac

We can clearly see that the accuracy of the model on test set is just $65.8\%$ on a sub-training set of $384$ examples (3 batches), even though the training accuracies are perfect ($100\%$)

### Problem 3

Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaludation, otherwise our evaluation results would be stochastic as well.

In [29]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    out_weights = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    drop1 = tf.nn.dropout(layer1_train, 0.5)
    logits = tf.matmul(drop1, out_weights) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization
        0.001 * tf.nn.l2_loss(hidden_weights) + 
        0.001 * tf.nn.l2_loss(out_weights))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(
        tf.matmul(layer1_valid, out_weights) + out_biases)
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, out_weights) + out_biases)

In [32]:
num_steps = 40001

# Use a smaller train_dataset
train_smaller = train_dataset[:384, :]
labels_smaller = train_labels[:384, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_smaller.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_smaller[offset:(offset + batch_size), :]
        batch_labels = labels_smaller[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 4000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 845.106079
Minibatch accuracy: 6.2%
Validation accuracy: 19.9%
Minibatch loss at step 4000: 228.378677
Minibatch accuracy: 99.2%
Validation accuracy: 74.2%
Minibatch loss at step 8000: 165.671066
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Minibatch loss at step 12000: 120.384972
Minibatch accuracy: 99.2%
Validation accuracy: 74.0%
Minibatch loss at step 16000: 87.353783
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Minibatch loss at step 20000: 63.430855
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 24000: 46.059589
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 28000: 33.445766
Minibatch accuracy: 100.0%
Validation accuracy: 74.6%
Minibatch loss at step 32000: 24.286348
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 36000: 17.635323
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 40000: 12.805796
Minibatch accu